In [1]:
import pandas as pd
from jieba import jieba
#import custom dictionary
jieba.load_userdict("dict.txt")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Administrator\AppData\Local\Temp\jieba.cache
Loading model cost 0.668 seconds.
Prefix dict has been built succesfully.


In [2]:
dict = pd.read_table('dict.txt',header=None,encoding='utf-8')
print(dict.head())

          0
0      奥迪A3
1     奥迪A4L
2     奥迪A6L
3  奥迪A6L新能源
4     奥迪Q2L


In [3]:
comp_train_data = pd.read_csv('train.csv')

In [4]:
len(comp_train_data)

9947

In [5]:
print(comp_train_data.head())

         content_id                                            content  \
0  vUXizsqexyZVRdFH           因为森林人即将换代，这套系统没必要装在一款即将换代的车型上，因为肯定会影响价格。   
1  4QroPd9hNfnCHVt7      四驱价格貌似挺高的，高的可以看齐XC60了，看实车前脸有点违和感。不过大众的车应该不会差。   
2  QmqJ2AvM5GplaRyz  斯柯达要说质量，似乎比大众要好一点，价格也低一些，用料完全一样。我听说过野帝，但没听说过你说...   
3  KMT1gFJiU4NWrVDn           这玩意都是给有钱任性又不懂车的土豪用的，这价格换一次我妹夫EP020可以换三锅了   
4  nVIlGd5yMmc37t1o                            17价格忒高，估计也就是14-15左右。      

  subject  sentiment_value sentiment_word  
0      价格                0             影响  
1      价格               -1              高  
2      价格                1              低  
3      价格               -1           有钱任性  
4      价格               -1              高  


In [6]:
comp_train_data['content'].head()

0             因为森林人即将换代，这套系统没必要装在一款即将换代的车型上，因为肯定会影响价格。
1        四驱价格貌似挺高的，高的可以看齐XC60了，看实车前脸有点违和感。不过大众的车应该不会差。
2    斯柯达要说质量，似乎比大众要好一点，价格也低一些，用料完全一样。我听说过野帝，但没听说过你说...
3             这玩意都是给有钱任性又不懂车的土豪用的，这价格换一次我妹夫EP020可以换三锅了
4                              17价格忒高，估计也就是14-15左右。   
Name: content, dtype: object

In [7]:
train = comp_train_data.drop(['content_id','sentiment_word'],axis=1)

In [8]:
subjects = {'动力':0,'价格':1,'内饰':2,'配置':3,'安全性':4,'外观':5,'操控':6,'油耗':7,'空间':8,'舒适性':9}

In [9]:
for key, value in subjects.items():
    train['subject'].replace(to_replace=key, value=value,inplace=True)
    


In [10]:
train.head()

,content,subject,sentiment_value
0,因为森林人即将换代，这套系统没必要装在一款即将换代的车型上，因为肯定会影响价格。,1,0
1,四驱价格貌似挺高的，高的可以看齐XC60了，看实车前脸有点违和感。不过大众的车应该不会差。,1,-1
2,斯柯达要说质量，似乎比大众要好一点，价格也低一些，用料完全一样。我听说过野帝，但没听说过你说...,1,1
3,这玩意都是给有钱任性又不懂车的土豪用的，这价格换一次我妹夫EP020可以换三锅了,1,-1
4,17价格忒高，估计也就是14-15左右。,1,-1


In [11]:
#load big data
big_data = pd.read_csv('training_data.csv',usecols=['content', 'subject'])
big_data = pd.DataFrame(big_data)
print('big_data shape = {}'.format(big_data.shape))

big_data shape = (237178, 2)


In [12]:
temp_1 = train[['content', 'subject']]
subject_train = pd.concat([big_data, temp_1])
subject_train = subject_train.drop_duplicates(subset=['content'], keep='first', inplace=False)
bigdata_train = pd.DataFrame(subject_train, columns=['content', 'subject'])
print('Total_shape = {}'.format(bigdata_train.shape))

Total_shape = (172534, 2)


In [13]:
bigdata_train.head()

,content,subject
0,这个动力表现略显富余，对于日常超车这样的事情简直不要太轻松了。油门反应较为迅速，加上操控表现...,0
1,2.0T的动力很给力，只要舍得踩油门，推背感说来就来，加速很迅速。车子在换挡的时候也很积极，...,0
2,动力方面还是比较满意的，1.4T高功率，最大马力150，没有大家想象的那么差。虽然车小比较轻...,0
3,1.4T高功率的配置，动力可以说非常畅快了，涡轮的介入也非常的及时。我起步喜欢地板油，那种推...,0
4,发动机动力不错，比一般车子强多了，油门踩下去动力就上来了反应很快捷一点都不肉，怠速时非常的稳...,0


In [14]:
#中文分词测试
print('原文：{}'.format(bigdata_train['content'].iloc[341]))
text = jieba.cut(bigdata_train['content'].iloc[341])
for word in text:
    print(word)

原文：动力配合较轻的车重比较合理，一个人的时候得心应手。开空调有些力不从心，提高转速是唯一的选择，满载也勉强够用。
动力
配合
较轻
的
车重
比较
合理
，
一个
人
的
时候
得心应手
。
开
空调
有些
力不从心
，
提高
转速
是
唯一
的
选择
，
满载
也
勉强
够用
。


In [3]:
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from gensim.models import word2vec

C:\Users\Administrator\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [4]:
#载入预训练好的word2Vec模型
model = word2vec.Word2Vec.load('word2vec.model')

2018-09-24 15:25:41,073 : INFO : loading Word2Vec object from word2vec.model
2018-09-24 15:25:41,518 : INFO : loading wv recursively from word2vec.model.wv.* with mmap=None
2018-09-24 15:25:41,518 : INFO : setting ignored attribute vectors_norm to None
2018-09-24 15:25:41,518 : INFO : loading vocabulary recursively from word2vec.model.vocabulary.* with mmap=None
2018-09-24 15:25:41,518 : INFO : loading trainables recursively from word2vec.model.trainables.* with mmap=None
2018-09-24 15:25:41,518 : INFO : setting ignored attribute cum_table to None
2018-09-24 15:25:41,523 : INFO : loaded word2vec.model


In [17]:
#切词转换成训练好的word2vec字典的index
import re

def convert_data_to_index(model, content):
    index_data = []
    for word in content:
        if word in model.wv:
            index_data.append(model.wv.vocab[word].index + 1)
    return index_data

def word_to_idx(text, label):
    train_set = []
    for index, content in enumerate(text):
        sentence = []
        contents = re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？?、~@#￥%……&*（）]+", "",content)
        for word in jieba.cut(contents):
            sentence.append(word)
        idx_sentence = convert_data_to_index(model,sentence)
        train_set.append([idx_sentence, label.iloc[index]])
    return train_set



In [18]:
training_set = word_to_idx(bigdata_train['content'], bigdata_train['subject'])

In [19]:
training_set = pd.DataFrame(training_set, columns=['content','subject'])
print(training_set.head())

                                             content  subject
0  [31, 22, 213, 1026, 2944, 144, 422, 154, 152, ...        0
1  [1125, 1, 22, 4, 63, 419, 215, 311, 376, 513, ...        0
2  [22, 130, 9, 272, 1, 937, 5868, 456, 939, 1497...        0
3  [937, 5868, 1, 54, 22, 18, 29, 32, 6618, 2, 31...        0
4  [71, 22, 19, 51, 48, 92, 2676, 2, 167, 2702, 2...        0


In [20]:
#去除只有0内容的评论
import numpy as np
from collections import Counter

int_trains = np.array(training_set['content'])
review_length = Counter([len(x) for x in int_trains])
print('0内容评论有：{}条'.format(review_length[0]))

0内容评论有：14条


In [21]:
#Filter out the 0 length review
non_zero_idx = [ii for ii, review in enumerate(int_trains) if len(review)!=0]
reviews_input = [int_trains[i] for i in non_zero_idx]
labels = np.array([training_set['subject'].iloc[i] for i in non_zero_idx])

In [5]:
seq_len = 80

def content_to_seq(contents, seq_len):
    features_forward = np.zeros((len(contents), seq_len), dtype=int)
    features_backward = np.zeros((len(contents), seq_len), dtype=int)
    for i, review in enumerate(contents):
        features_forward[i, -len(review):] = np.array(review)[:seq_len]
        features_backward[i, -len(review):] = np.array(review)[:seq_len][::-1]
    return features_forward, features_backward



In [ ]:
features_forward, features_backward = content_to_seq(reviews_input, seq_len)

In [23]:
#将label OneHot化
from keras.utils import to_categorical
OneHot_labels = to_categorical(labels, num_classes=10)

Using TensorFlow backend.


In [24]:
from sklearn.model_selection import train_test_split

X_train_F, X_test_F, y_train_F, y_test_F = train_test_split(features_forward, OneHot_labels, test_size=0.30,shuffle=True, random_state=100)
X_train_B, X_test_B, y_train_B, y_test_B = train_test_split(features_backward, OneHot_labels, test_size=0.30,shuffle=True, random_state=100)

In [25]:
print(X_test_B[618],y_test_B[618])
print(X_test_F[618],y_test_F[618])

test_f = X_test_F[618]
test_b = X_test_B[618]

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0  851  632    9 1546 3240    3  179   22] [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0   22  179    3 3240 1546    9  632  851] [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [26]:
batch_size = 512
state_size = 300
lstm_layers = 2
max_grad = 5
learning_rate = 0.001

In [27]:
from BiLstm_model import BiLstm_Model

In [28]:
rnn_model = BiLstm_Model(batch_size,seq_len,state_size,lstm_layers,max_grad,learning_rate)

2018-09-24 08:47:04,223 : INFO : loading Word2Vec object from word2vec.model
2018-09-24 08:47:04,631 : INFO : loading wv recursively from word2vec.model.wv.* with mmap=None
2018-09-24 08:47:04,631 : INFO : setting ignored attribute vectors_norm to None
2018-09-24 08:47:04,631 : INFO : loading vocabulary recursively from word2vec.model.vocabulary.* with mmap=None
2018-09-24 08:47:04,631 : INFO : loading trainables recursively from word2vec.model.trainables.* with mmap=None
2018-09-24 08:47:04,631 : INFO : setting ignored attribute cum_table to None
2018-09-24 08:47:04,643 : INFO : loaded word2vec.model


In [29]:
import tensorflow as tf

In [30]:
epochs = 102

    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    iteration = 1
    
    accuarcy_hist = []
    saver=tf.train.Saver(max_to_keep=3)
    
    max_acc=0
        
    for i_epoch in range(epochs):
        
        
        for f, b, y in rnn_model.get_batches(X_train_F, X_train_B, y_train_F):
           
            feed_dict = {rnn_model.forward_placeholder:f, rnn_model.backward_placeholder:b,rnn_model.labels_placeholder:y
                    ,rnn_model.dropout_placeholder:0.5}
            _, loss, pred= sess.run([rnn_model.train_op, rnn_model.loss, rnn_model.pred], feed_dict=feed_dict)
                
            if iteration%10==0:
                mean_loss = np.mean(loss)
                #training_loss.append(mean_loss)
                print("Epoch: {}/{}".format(i_epoch, epochs),
                      "Iteration: {}".format(iteration),
                        "Train loss: {:.3f}".format(mean_loss))
                               
               
            if iteration%100==0:
                test_acc = []
                
                for pred_label, label in zip(pred, y):
                    if np.argmax(pred_label) == np.argmax(label):
                        test_acc.append(1)
                    else:
                        test_acc.append(0)
                mean_acc = np.mean(test_acc)
             
                print("Test_set acc: {:.3f}".format(mean_acc))
            iteration +=1
            
        if i_epoch%10==1:
            test_accuracy = []
            
            for f, b, y in rnn_model.get_batches(X_test_F, X_test_B, y_test_F):
                
                feed_dict = {rnn_model.forward_placeholder:f, rnn_model.backward_placeholder:b,rnn_model.labels_placeholder:y
                    ,rnn_model.dropout_placeholder:1}
                loss, pred= sess.run([rnn_model.loss, rnn_model.pred], feed_dict=feed_dict)
                val_acc = []
                for pred_label, label in zip(pred, y):
                    if np.argmax(pred_label) == np.argmax(label):
                        val_acc.append(1)
                    else:
                        val_acc.append(0)
                mean_acc = np.mean(val_acc)
                
                test_accuracy.append(mean_acc)
            result = np.mean(test_accuracy)
           
            
            if result>max_acc:
                f=open('ckpt/acc.txt','w')
                f.write(str(i_epoch+1)+', val_acc: '+str(result)+'\n')
                f.close()  
                max_acc=result
                saver.save(sess,'ckpt/mnist.ckpt',global_step=i_epoch+1)
             
            print("Val_set acc: {:.3f}".format(result))
                
            accuarcy_hist.append(result)       
    
                
                

Epoch: 0/102 Iteration: 10 Train loss: 2.327
Epoch: 0/102 Iteration: 20 Train loss: 2.294
Epoch: 0/102 Iteration: 30 Train loss: 2.251
Epoch: 0/102 Iteration: 40 Train loss: 2.254
Epoch: 0/102 Iteration: 50 Train loss: 2.234
Epoch: 0/102 Iteration: 60 Train loss: 2.200
Epoch: 0/102 Iteration: 70 Train loss: 2.192
Epoch: 0/102 Iteration: 80 Train loss: 2.192
Epoch: 0/102 Iteration: 90 Train loss: 2.160
Epoch: 0/102 Iteration: 100 Train loss: 2.149
Test_set acc: 0.279
Epoch: 0/102 Iteration: 110 Train loss: 2.141
Epoch: 0/102 Iteration: 120 Train loss: 2.113
Epoch: 0/102 Iteration: 130 Train loss: 2.083
Epoch: 0/102 Iteration: 140 Train loss: 2.109
Epoch: 0/102 Iteration: 150 Train loss: 2.090
Epoch: 0/102 Iteration: 160 Train loss: 2.069
Epoch: 0/102 Iteration: 170 Train loss: 2.059
Epoch: 0/102 Iteration: 180 Train loss: 2.063
Epoch: 0/102 Iteration: 190 Train loss: 2.027
Epoch: 0/102 Iteration: 200 Train loss: 2.030
Test_set acc: 0.436
Epoch: 0/102 Iteration: 210 Train loss: 2.033
Epo

Epoch: 7/102 Iteration: 1710 Train loss: 1.727
Epoch: 7/102 Iteration: 1720 Train loss: 1.726
Epoch: 7/102 Iteration: 1730 Train loss: 1.738
Epoch: 7/102 Iteration: 1740 Train loss: 1.732
Epoch: 7/102 Iteration: 1750 Train loss: 1.728
Epoch: 7/102 Iteration: 1760 Train loss: 1.732
Epoch: 7/102 Iteration: 1770 Train loss: 1.708
Epoch: 7/102 Iteration: 1780 Train loss: 1.727
Epoch: 7/102 Iteration: 1790 Train loss: 1.722
Epoch: 7/102 Iteration: 1800 Train loss: 1.706
Test_set acc: 0.785
Epoch: 7/102 Iteration: 1810 Train loss: 1.725
Epoch: 7/102 Iteration: 1820 Train loss: 1.721
Epoch: 7/102 Iteration: 1830 Train loss: 1.713
Epoch: 7/102 Iteration: 1840 Train loss: 1.734
Epoch: 7/102 Iteration: 1850 Train loss: 1.711
Epoch: 7/102 Iteration: 1860 Train loss: 1.701
Epoch: 7/102 Iteration: 1870 Train loss: 1.719
Epoch: 7/102 Iteration: 1880 Train loss: 1.733
Epoch: 8/102 Iteration: 1890 Train loss: 1.730
Epoch: 8/102 Iteration: 1900 Train loss: 1.715
Test_set acc: 0.793
Epoch: 8/102 Iterati

Epoch: 14/102 Iteration: 3360 Train loss: 1.679
Epoch: 14/102 Iteration: 3370 Train loss: 1.692
Epoch: 14/102 Iteration: 3380 Train loss: 1.666
Epoch: 14/102 Iteration: 3390 Train loss: 1.681
Epoch: 14/102 Iteration: 3400 Train loss: 1.686
Test_set acc: 0.799
Epoch: 14/102 Iteration: 3410 Train loss: 1.669
Epoch: 14/102 Iteration: 3420 Train loss: 1.661
Epoch: 14/102 Iteration: 3430 Train loss: 1.691
Epoch: 14/102 Iteration: 3440 Train loss: 1.671
Epoch: 14/102 Iteration: 3450 Train loss: 1.688
Epoch: 14/102 Iteration: 3460 Train loss: 1.690
Epoch: 14/102 Iteration: 3470 Train loss: 1.680
Epoch: 14/102 Iteration: 3480 Train loss: 1.665
Epoch: 14/102 Iteration: 3490 Train loss: 1.691
Epoch: 14/102 Iteration: 3500 Train loss: 1.662
Test_set acc: 0.809
Epoch: 14/102 Iteration: 3510 Train loss: 1.679
Epoch: 14/102 Iteration: 3520 Train loss: 1.672
Epoch: 15/102 Iteration: 3530 Train loss: 1.664
Epoch: 15/102 Iteration: 3540 Train loss: 1.672
Epoch: 15/102 Iteration: 3550 Train loss: 1.672


Epoch: 21/102 Iteration: 5010 Train loss: 1.644
Epoch: 21/102 Iteration: 5020 Train loss: 1.671
Epoch: 21/102 Iteration: 5030 Train loss: 1.661
Epoch: 21/102 Iteration: 5040 Train loss: 1.662
Epoch: 21/102 Iteration: 5050 Train loss: 1.678
Epoch: 21/102 Iteration: 5060 Train loss: 1.659
Epoch: 21/102 Iteration: 5070 Train loss: 1.671
Epoch: 21/102 Iteration: 5080 Train loss: 1.666
Epoch: 21/102 Iteration: 5090 Train loss: 1.665
Epoch: 21/102 Iteration: 5100 Train loss: 1.673
Test_set acc: 0.809
Epoch: 21/102 Iteration: 5110 Train loss: 1.677
Epoch: 21/102 Iteration: 5120 Train loss: 1.665
Epoch: 21/102 Iteration: 5130 Train loss: 1.673
Epoch: 21/102 Iteration: 5140 Train loss: 1.649
Epoch: 21/102 Iteration: 5150 Train loss: 1.644
Epoch: 21/102 Iteration: 5160 Train loss: 1.660
Epoch: 21/102 Iteration: 5170 Train loss: 1.681
Val_set acc: 0.847
Epoch: 22/102 Iteration: 5180 Train loss: 1.672
Epoch: 22/102 Iteration: 5190 Train loss: 1.672
Epoch: 22/102 Iteration: 5200 Train loss: 1.682
T

Epoch: 28/102 Iteration: 6650 Train loss: 1.656
Epoch: 28/102 Iteration: 6660 Train loss: 1.669
Epoch: 28/102 Iteration: 6670 Train loss: 1.643
Epoch: 28/102 Iteration: 6680 Train loss: 1.659
Epoch: 28/102 Iteration: 6690 Train loss: 1.665
Epoch: 28/102 Iteration: 6700 Train loss: 1.660
Test_set acc: 0.797
Epoch: 28/102 Iteration: 6710 Train loss: 1.650
Epoch: 28/102 Iteration: 6720 Train loss: 1.679
Epoch: 28/102 Iteration: 6730 Train loss: 1.639
Epoch: 28/102 Iteration: 6740 Train loss: 1.657
Epoch: 28/102 Iteration: 6750 Train loss: 1.668
Epoch: 28/102 Iteration: 6760 Train loss: 1.657
Epoch: 28/102 Iteration: 6770 Train loss: 1.649
Epoch: 28/102 Iteration: 6780 Train loss: 1.675
Epoch: 28/102 Iteration: 6790 Train loss: 1.648
Epoch: 28/102 Iteration: 6800 Train loss: 1.669
Test_set acc: 0.791
Epoch: 28/102 Iteration: 6810 Train loss: 1.655
Epoch: 29/102 Iteration: 6820 Train loss: 1.643
Epoch: 29/102 Iteration: 6830 Train loss: 1.665
Epoch: 29/102 Iteration: 6840 Train loss: 1.646


...

Epoch: 69/102 Iteration: 16370 Train loss: 1.642
Epoch: 69/102 Iteration: 16380 Train loss: 1.648
Epoch: 69/102 Iteration: 16390 Train loss: 1.641
Epoch: 69/102 Iteration: 16400 Train loss: 1.642
Test_set acc: 0.818
Epoch: 69/102 Iteration: 16410 Train loss: 1.659
Epoch: 69/102 Iteration: 16420 Train loss: 1.628
Epoch: 69/102 Iteration: 16430 Train loss: 1.634
Epoch: 69/102 Iteration: 16440 Train loss: 1.631
Epoch: 69/102 Iteration: 16450 Train loss: 1.664
Epoch: 70/102 Iteration: 16460 Train loss: 1.661
Epoch: 70/102 Iteration: 16470 Train loss: 1.647
Epoch: 70/102 Iteration: 16480 Train loss: 1.662
Epoch: 70/102 Iteration: 16490 Train loss: 1.648
Epoch: 70/102 Iteration: 16500 Train loss: 1.630
Test_set acc: 0.830
Epoch: 70/102 Iteration: 16510 Train loss: 1.659
Epoch: 70/102 Iteration: 16520 Train loss: 1.647
Epoch: 70/102 Iteration: 16530 Train loss: 1.657
Epoch: 70/102 Iteration: 16540 Train loss: 1.632
Epoch: 70/102 Iteration: 16550 Train loss: 1.656
Epoch: 70/102 Iteration: 1656

Epoch: 76/102 Iteration: 17980 Train loss: 1.652
Epoch: 76/102 Iteration: 17990 Train loss: 1.616
Epoch: 76/102 Iteration: 18000 Train loss: 1.650
Test_set acc: 0.803
Epoch: 76/102 Iteration: 18010 Train loss: 1.630
Epoch: 76/102 Iteration: 18020 Train loss: 1.658
Epoch: 76/102 Iteration: 18030 Train loss: 1.648
Epoch: 76/102 Iteration: 18040 Train loss: 1.655
Epoch: 76/102 Iteration: 18050 Train loss: 1.639
Epoch: 76/102 Iteration: 18060 Train loss: 1.661
Epoch: 76/102 Iteration: 18070 Train loss: 1.626
Epoch: 76/102 Iteration: 18080 Train loss: 1.658
Epoch: 76/102 Iteration: 18090 Train loss: 1.639
Epoch: 77/102 Iteration: 18100 Train loss: 1.629
Test_set acc: 0.834
Epoch: 77/102 Iteration: 18110 Train loss: 1.649
Epoch: 77/102 Iteration: 18120 Train loss: 1.634
Epoch: 77/102 Iteration: 18130 Train loss: 1.638
Epoch: 77/102 Iteration: 18140 Train loss: 1.650
Epoch: 77/102 Iteration: 18150 Train loss: 1.677
Epoch: 77/102 Iteration: 18160 Train loss: 1.642
Epoch: 77/102 Iteration: 1817

Epoch: 83/102 Iteration: 19590 Train loss: 1.636
Epoch: 83/102 Iteration: 19600 Train loss: 1.647
Test_set acc: 0.805
Epoch: 83/102 Iteration: 19610 Train loss: 1.647
Epoch: 83/102 Iteration: 19620 Train loss: 1.650
Epoch: 83/102 Iteration: 19630 Train loss: 1.639
Epoch: 83/102 Iteration: 19640 Train loss: 1.658
Epoch: 83/102 Iteration: 19650 Train loss: 1.638
Epoch: 83/102 Iteration: 19660 Train loss: 1.652
Epoch: 83/102 Iteration: 19670 Train loss: 1.642
Epoch: 83/102 Iteration: 19680 Train loss: 1.636
Epoch: 83/102 Iteration: 19690 Train loss: 1.626
Epoch: 83/102 Iteration: 19700 Train loss: 1.668
Test_set acc: 0.801
Epoch: 83/102 Iteration: 19710 Train loss: 1.628
Epoch: 83/102 Iteration: 19720 Train loss: 1.633
Epoch: 83/102 Iteration: 19730 Train loss: 1.627
Epoch: 83/102 Iteration: 19740 Train loss: 1.654
Epoch: 84/102 Iteration: 19750 Train loss: 1.646
Epoch: 84/102 Iteration: 19760 Train loss: 1.649
Epoch: 84/102 Iteration: 19770 Train loss: 1.658
Epoch: 84/102 Iteration: 1978

Epoch: 90/102 Iteration: 21200 Train loss: 1.630
Test_set acc: 0.828
Epoch: 90/102 Iteration: 21210 Train loss: 1.664
Epoch: 90/102 Iteration: 21220 Train loss: 1.634
Epoch: 90/102 Iteration: 21230 Train loss: 1.662
Epoch: 90/102 Iteration: 21240 Train loss: 1.623
Epoch: 90/102 Iteration: 21250 Train loss: 1.651
Epoch: 90/102 Iteration: 21260 Train loss: 1.659
Epoch: 90/102 Iteration: 21270 Train loss: 1.638
Epoch: 90/102 Iteration: 21280 Train loss: 1.631
Epoch: 90/102 Iteration: 21290 Train loss: 1.656
Epoch: 90/102 Iteration: 21300 Train loss: 1.625
Test_set acc: 0.828
Epoch: 90/102 Iteration: 21310 Train loss: 1.637
Epoch: 90/102 Iteration: 21320 Train loss: 1.653
Epoch: 90/102 Iteration: 21330 Train loss: 1.649
Epoch: 90/102 Iteration: 21340 Train loss: 1.637
Epoch: 90/102 Iteration: 21350 Train loss: 1.659
Epoch: 90/102 Iteration: 21360 Train loss: 1.633
Epoch: 90/102 Iteration: 21370 Train loss: 1.666
Epoch: 90/102 Iteration: 21380 Train loss: 1.647
Epoch: 91/102 Iteration: 2139

Epoch: 97/102 Iteration: 22810 Train loss: 1.633
Epoch: 97/102 Iteration: 22820 Train loss: 1.629
Epoch: 97/102 Iteration: 22830 Train loss: 1.632
Epoch: 97/102 Iteration: 22840 Train loss: 1.657
Epoch: 97/102 Iteration: 22850 Train loss: 1.662
Epoch: 97/102 Iteration: 22860 Train loss: 1.653
Epoch: 97/102 Iteration: 22870 Train loss: 1.632
Epoch: 97/102 Iteration: 22880 Train loss: 1.634
Epoch: 97/102 Iteration: 22890 Train loss: 1.628
Epoch: 97/102 Iteration: 22900 Train loss: 1.642
Test_set acc: 0.826
Epoch: 97/102 Iteration: 22910 Train loss: 1.647
Epoch: 97/102 Iteration: 22920 Train loss: 1.645
Epoch: 97/102 Iteration: 22930 Train loss: 1.647
Epoch: 97/102 Iteration: 22940 Train loss: 1.630
Epoch: 97/102 Iteration: 22950 Train loss: 1.636
Epoch: 97/102 Iteration: 22960 Train loss: 1.632
Epoch: 97/102 Iteration: 22970 Train loss: 1.632
Epoch: 97/102 Iteration: 22980 Train loss: 1.646
Epoch: 97/102 Iteration: 22990 Train loss: 1.676
Epoch: 97/102 Iteration: 23000 Train loss: 1.630


In [6]:
#读比赛测试数据内容
import numpy as np
import pandas as pd
competition_test = pd.read_csv('test_public.csv')
competition_test.head()


,content_id,content
0,XuPwKCnA2fqNh5vm,欧蓝德，价格便宜，森林人太贵啦！
1,2jNbDn85goX3IuPE,楼主什么时候提的车，南昌优惠多少啊
2,hLgEADQ8sUnvGFK9,吉林，2.5优惠20000，送三年九次保养，贴膜
3,nZmM7LQsfr03wUaz,便宜2万的豪华特装，实用配制提升，优惠还给力，确实划算。
4,pwd8MnrthDqLZafe,如果实在想买就等车展期间，优惠2万，我24.98万入的2.5豪


In [7]:
import re

def convert_data_to_index(model, content):
    index_data = []
    for word in content:
        if word in model.wv:
            index_data.append(model.wv.vocab[word].index + 1)
    return index_data

def word_to_idx(text):
    train_set = []
    for index, content in enumerate(text):
        sentence = []
        contents = re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？?、~@#￥%……&*（）]+", "",content)
        for word in jieba.cut(contents):
            sentence.append(word)
        idx_sentence = convert_data_to_index(model,sentence)
        train_set.append(np.array(idx_sentence))
    return train_set


In [8]:
#数据清洗并生产features作为训练输入
test_set = word_to_idx(competition_test['content'])
test_set = np.array(test_set)
test_forward, test_backward = content_to_seq(test_set, seq_len)

In [9]:
import tensorflow as tf
from BiLstm_model import BiLstm_Model

In [10]:
batch_size = 1
state_size = 500
lstm_layers = 2
max_grad = 5
learning_rate = 0.001

In [11]:
rnn_model = BiLstm_Model(1,seq_len,state_size,lstm_layers,max_grad,learning_rate)

2018-09-24 15:26:10,421 : INFO : loading Word2Vec object from word2vec.model
2018-09-24 15:26:10,819 : INFO : loading wv recursively from word2vec.model.wv.* with mmap=None
2018-09-24 15:26:10,819 : INFO : setting ignored attribute vectors_norm to None
2018-09-24 15:26:10,819 : INFO : loading vocabulary recursively from word2vec.model.vocabulary.* with mmap=None
2018-09-24 15:26:10,819 : INFO : loading trainables recursively from word2vec.model.trainables.* with mmap=None
2018-09-24 15:26:10,819 : INFO : setting ignored attribute cum_table to None
2018-09-24 15:26:10,835 : INFO : loaded word2vec.model


In [12]:
with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    saver=tf.train.Saver(max_to_keep=3)
    model_file=tf.train.latest_checkpoint('ckpt/')
    saver.restore(sess,model_file)
    

    pred_labels = []
    
    for f, b in zip(test_backward, test_backward):
        f = np.reshape(f,(1,80))
        b = np.reshape(b,(1,80))
    
   
       
            
        
        feed_dict = {rnn_model.forward_placeholder:f, rnn_model.backward_placeholder:b
                    ,rnn_model.dropout_placeholder:1}
        pred= sess.run([rnn_model.pred], feed_dict=feed_dict)

        pred_labels.append(np.argmax(pred))

2018-09-24 15:26:22,941 : INFO : Restoring parameters from ckpt/mnist.ckpt-12


In [13]:
test_labels = pd.DataFrame(data=pred_labels, columns=['label'])

final_test = pd.concat([competition_test, test_labels],axis=1)
final_test = final_test.drop(['content_id'],axis=1)

In [14]:
subjects = {'动力':0,'价格':1,'内饰':2,'配置':3,'安全性':4,'外观':5,'操控':6,'油耗':7,'空间':8,'舒适性':9}
for key, value in subjects.items():
    final_test['label'].replace(to_replace=value, value=key,inplace=True)
    

In [15]:
print(final_test[860:880])

                                               content label
860                                        我打开空调，后排只喊冷   舒适性
861  不过开起来是20的油比30的油，油耗稍微低那么一点点。。。。。。。加上车基本就是主要上下班和...    油耗
862                                      车不错，价格再靠谱点就好了    价格
863                                直接买 价格车况好的话13以内都没问题    价格
864                   不过和神丁聊改车，买的改件都是精品，要提升操控就操控，提升动力，    价格
865                       森就是好开，小众，能接受就买。考虑配置性价比，还是算了。    价格
866  楼主，根据我的亲身经历，我的判断是机油加多了，你可以看看机油尺 是否超过了上限，如果是的，随...    动力
867  一个买菜车，烧机油，到处响，座椅才两年半就开裂。不知道何来性能。发动机过七万公里，声音就像拖...    动力
868                                 大家都说导航是最不能忍的，还不如不加    配置
869                           原来导航的应该就是本来跟车的吧！你车是什么版本？    配置
870  换回油饼原配密封圈，把管固定好，还是有点点渗，我发现垫高油饼上靠近密封圈处有点小小的不平，不...   舒适性
871        其实我也纠结这两款，为了装b，忍受12秒的加速，花多5万块，最后感觉还是新crv好点。    动力
872                                 去看看国产法式豪华DS的维修价格吧！    价格
873  正常16左右行情价，18估计难！2.5T版本也就比2.5版本多卖个三万块钱，其实对于当年新车...    价格
874                                       不到19万 这价格无敌了    价格
875                     